#### Programas principales


In [17]:
import subprocess
import json

api_response = []
url = f'https://radiocut.fm/api/search/?type=cut&page=160494'
command = f'curl --request GET --url {url}'
result = subprocess.run(command, shell=True, capture_output=True, text=True, encoding='utf-8')
 
# Parse JSON response
json_data = json.loads(result.stdout)

# Extend the list with new results
api_response.extend(json_data['results'])

print(api_response)

[{'type': 'AudioCut', 'value': {'episode': None, 'description': '"Ojo Negro".\r\nLa primera entrevista de Alberto Fernández con un medio argentino y sus excusas.\r\nAlberto Fernández le dio su versión de los hechos al Cohete en una entrevista de una hora y cuarto en la mañana del viernes 9, con la condición de que no se publique antes de la otra entrevista que concedió, al diario español El País, que debería salir hoy. El ex Presidente alega que el hematoma no se debe a un golpe, sino a un tratamiento estético contra las arrugas. Los medios que siguen cada minuto del caso, recuerdan que también en 2021 Fernández primero negó que se hubiera producido el festejo en Olivos, luego puso en duda la autenticidad de la foto y terminó por incriminar a su pareja.\r\nEl editorial de Horacio Verbitsky del 11/08/2024 en El Cohete a la Luna.', 'title': 'Horacio Verbitsky - Ojo Negro - El Cohete a la Luna', 'radio_obj': {'slug': 'madres', 'name': 'AM 530 - Somos Radio', 'webpage': 'https://am530somos

In [9]:
import os
from dotenv import load_dotenv
import subprocess
import json
from collections import defaultdict
from datetime import datetime, timedelta

load_dotenv()

radiocut_api_key = os.getenv("RADIOCUT_API_KEY")

# Generar URLs para páginas 1 a 30
urls = [f'https://radiocut.fm/api/search/?type=cut&page={i}' for i in range(160492, 160493)]

headers = {
    "Authorization": f"Token {radiocut_api_key}",
    "accept": "application/json; charset=utf-8"
}

header_str = " ".join([f'--header \"{key}: {value}\"' for key, value in headers.items()])

resultados = []

for url in urls:
    command = f'curl --request GET --url {url} {header_str}'
    result = subprocess.run(command, shell=True, capture_output=True, text=True, encoding='utf-8')
    json_data = json.loads(result.stdout)
    resultados.extend(json_data['results'])  # Recopilar todos los resultados

print(resultados)
replicados = []
user_views = defaultdict(int)
user_recortes = defaultdict(int)
radio_recortes = defaultdict(int)
programa_recortes = defaultdict(int)

# Usar un set para identificar duplicados
seen = set()

duraciones = []

for item in resultados:
    titulo = item['value']['title']
    programa = item['value']['show']
    radio = item['value']['radio_obj']['name']
    empieza = item['value']['start']
    duracion = item['value']['length']
    descripcion = item['value']['description']
    usuario = item['value']['owner']['username']
    reproducciones = item['value']['play_count']
    etiquetas = item['value']['tags']
    slug = item['value']['slug']

    # Convertir 'empieza' a formato HH:MM:SS
    empieza_datetime = datetime.fromisoformat(empieza.replace('Z', '+00:00'))
    empieza_formatted = empieza_datetime.strftime('%H:%M:%S')

    # Convertir duración de segundos a formato HH:MM:SS
    def format_duration(seconds):
        return str(timedelta(seconds=seconds))

    duracion_formatted = format_duration(duracion)

    # Agregar la duración a la lista para cálculos estadísticos
    duraciones.append(duracion)

    # Generar URL con el título
    sanitized_title = slug
    generated_url = f"https://radiocut.fm/audiocut/{sanitized_title}"

    # Crear una clave única para el recorte
    unique_key = (titulo, programa, radio, empieza, duracion, descripcion, usuario, reproducciones, str(etiquetas))

    if unique_key not in seen:
        seen.add(unique_key)
        replicados.append({
            'titulo': titulo,
            'programa': programa,
            'radio': radio,
            'empieza': empieza_formatted,
            'duracion': duracion_formatted,
            'descripcion': descripcion,
            'usuario': usuario,
            'reproducciones': reproducciones,
            'etiquetas': etiquetas,
            'url': generated_url
        })

    # Agregar vistas y recortes por usuario
    user_views[usuario] += reproducciones
    user_recortes[usuario] += 1

    # Agregar recortes por radio y programa
    radio_recortes[radio] += 1
    programa_recortes[programa] += 1

# Ordenar por reproducciones para los 10 mejores usuarios
top_users_by_views = sorted(user_views.items(), key=lambda x: x[1], reverse=True)[:10]

# Ordenar por total de recortes para los 100 mejores usuarios
top_users_by_recortes = sorted(user_recortes.items(), key=lambda x: x[1], reverse=True)[:100]

# Obtener los 10 recortes con más vistas
sorted_replicados = sorted(replicados, key=lambda x: x['reproducciones'], reverse=True)
top_10_recortes = sorted_replicados[:10]

# Calcular estadísticas
duracion_max = max(duraciones) if duraciones else None
duracion_min = min(duraciones) if duraciones else None
promedio_duracion = sum(duraciones) / len(duraciones) if duraciones else None

duracion_max_formatted = format_duration(duracion_max) if duracion_max else None
duracion_min_formatted = format_duration(duracion_min) if duracion_min else None
promedio_duracion_formatted = format_duration(promedio_duracion) if promedio_duracion else None

# No es posible extraer años del campo 'empieza', por lo que este cálculo se omite

# Obtener las 5 radios y 5 programas con más recortes
top_5_radios = sorted(radio_recortes.items(), key=lambda x: x[1], reverse=True)[:5]
top_5_programas = sorted(programa_recortes.items(), key=lambda x: x[1], reverse=True)[:5]

# Imprimir resultados
print("Primeros 10 recortes con más vistas:")
print(json.dumps(top_10_recortes, indent=4, ensure_ascii=False))

print(f"\nCantidad de recortes evaluados: {len(replicados)}")
# No se imprime la cantidad de años evaluados

print(f"\nRecorte con más duración: {duracion_max_formatted}")
print(f"Promedio de duración de los recortes: {promedio_duracion_formatted}")
print(f"Recorte con menos duración: {duracion_min_formatted}")

print("\nTop 10 usuarios con más vistas:")
print(json.dumps(top_users_by_views, indent=4, ensure_ascii=False))

print("\nTop 100 usuarios con más recortes:")
print(json.dumps(top_users_by_recortes, indent=4, ensure_ascii=False))

print("\nTop 5 radios con más recortes:")
print(json.dumps(top_5_radios, indent=4, ensure_ascii=False))

print("\nTop 5 programas con más recortes:")
print(json.dumps(top_5_programas, indent=4, ensure_ascii=False))



[{'type': 'AudioCut', 'value': {'episode': None, 'description': '"Ojo Negro".\r\nLa primera entrevista de Alberto Fernández con un medio argentino y sus excusas.\r\nAlberto Fernández le dio su versión de los hechos al Cohete en una entrevista de una hora y cuarto en la mañana del viernes 9, con la condición de que no se publique antes de la otra entrevista que concedió, al diario español El País, que debería salir hoy. El ex Presidente alega que el hematoma no se debe a un golpe, sino a un tratamiento estético contra las arrugas. Los medios que siguen cada minuto del caso, recuerdan que también en 2021 Fernández primero negó que se hubiera producido el festejo en Olivos, luego puso en duda la autenticidad de la foto y terminó por incriminar a su pareja.\r\nEl editorial de Horacio Verbitsky del 11/08/2024 en El Cohete a la Luna.', 'title': 'Horacio Verbitsky - Ojo Negro - El Cohete a la Luna', 'radio_obj': {'slug': 'madres', 'name': 'AM 530 - Somos Radio', 'webpage': 'https://am530somos

In [98]:
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

In [102]:
print(resultados)

[{'type': 'AudioCut', 'value': {'episode': None, 'description': 'Argentina en la semana "Alberto Fernández"', 'title': 'Editorial Eduardo Aliverti 10/08/2024', 'radio_obj': {'slug': 'lared', 'name': 'La Red', 'webpage': 'https://www.lared.am/', 'audio_url': 'https://playerservices.streamtheworld.com/pls/LA_RED_AM910AAC.pls', 'time_zone': 'America/Buenos_Aires', 'logo': 'https://imgsvr.radiocut.site/get/crop/center/200/200/radio_logos/11/07/11070b07-0331-4561-98c2-8c012f818238.png', 'freq': '910.00', 'listen_mode': 'ondemand+live', 'band': 'AM', 'city': {'code': 'BUE', 'state': {'code': 'AR-C', 'country': {'code': 'AR', 'name': 'Argentina'}, 'name': 'Ciudad Autónoma de Buenos Aires'}, 'name': 'Ciudad de Buenos Aires'}, 'station_type': 'r', 'visibility': 'p', 'description_text': ''}, 'tags': [], 'show': 'marca-de-radio', 'show_obj': {'slug': 'marca-de-radio', 'name': 'Marca de Radio', 'description': 'Conduce:Eduardo Aliverti ', 'radio': 'lared', 'twitter': '', 'facebook': 'https://www.fa

In [104]:
def flatDictToTxt(dict_list, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for dictionary in dict_list:
            for key, value in dictionary.items():
                # Escribe cada llave y valor en el archivo
                file.write(f"{key}: {value}\n")  # Cambio a '\n' para nueva línea
            file.write("\n")  # Añade una línea en blanco entre diccionarios

# Ejemplo de uso

file_path = 'output.txt'

# Escritura del archivo
flatDictToTxt(resultados, file_path)

In [111]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_mistralai import MistralAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

mistral = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=api_key
)

# Cargar el archivo de texto
loader = TextLoader("output.txt", encoding="utf-8")
documents = loader.load()

# Dividir el texto en chunks
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)  # Redujimos el chunk_size
texts = text_splitter.split_documents(documents)

# Obtener embeddings y manejar errores
try:
    embeddings = mistral.embed_documents([text.page_content for text in texts])
    
    # Verificar si la respuesta contiene la clave 'data'
    if 'data' not in embeddings:
        raise ValueError(f"Formato de respuesta inesperado: {embeddings}")

    # Crear vectorstore usando FAISS
    vectorstore = FAISS.from_documents(texts, mistral)

except Exception as e:
    print(f"Error al obtener embeddings o al crear vectorstore: {e}")

Created a chunk of size 2221, which is longer than the specified 1024
Created a chunk of size 2290, which is longer than the specified 1024
Created a chunk of size 2631, which is longer than the specified 1024
Created a chunk of size 2250, which is longer than the specified 1024
Created a chunk of size 2916, which is longer than the specified 1024
Created a chunk of size 2398, which is longer than the specified 1024
Created a chunk of size 2273, which is longer than the specified 1024
Created a chunk of size 2228, which is longer than the specified 1024
Created a chunk of size 2631, which is longer than the specified 1024
Created a chunk of size 2601, which is longer than the specified 1024
Created a chunk of size 1544, which is longer than the specified 1024
Created a chunk of size 2568, which is longer than the specified 1024
Created a chunk of size 2221, which is longer than the specified 1024
Created a chunk of size 2290, which is longer than the specified 1024
Created a chunk of s

Error al obtener embeddings o al crear vectorstore: 'data'


In [101]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Define una interfaz retriever (ahí está el context)
retriever = vectorstore.as_retriever()
# Instancia el Chat
model = ChatMistralAI(api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Responde la siguiente pregunta basándote únicamente en el contexto provisto:

<context>
{context}
</context>

Question: {input}""")

# Crea una cadena de retrieval para responder a la pregunta
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "¿Cuáles son los veinte recortes con más reproducciones?"})
print(response["answer"])

No es posible proporcionar los veinte recortes con más reproducciones, ya que el contexto solo proporciona detalles de ocho recortes. Sin embargo, puedo decir cuáles son los ocho recortes del contexto, ordenados de mayor a menor número de reproducciones.

1. Turista denunció que la Policía Caminera de Córdoba lo quiso coimear y amenazó con matar a su hijo - 207 reproducciones
2. PROGRAMA COMPLETO DEL 10-08-24 - 56 reproducciones
3. Editorial Eduardo Aliverti 10/08/2024 - 871 reproducciones
4. Reportaje a Ricardo Aronskind - 446 reproducciones
5. Andrés Larroque en Radio 10 - 98 reproducciones
6. Luis Mario Vitette con Víctor Hugo Morales - 695 reproducciones
7. Yo ya sé - Adelanto de La Lógica del escorpión - 2457 reproducciones
8. El pase entre Víctor Hugo y Cynthia García - 4944 reproducciones


#### Búsqueda de un término en todos los tipos de objetos, radios, programas y recortes

In [22]:
url = 'https://radiocut.fm/api/search/?type=all&search_term=avion'

command = f'curl --request GET --url {url} {header_str}'

result = subprocess.run(command, shell=True, capture_output=True, text=True, encoding='utf-8')

json_data = json.loads(result.stdout)

print(json.dumps(json_data, indent=4))

{
    "count": 2430558,
    "next": "https://radiocut.fm/api/search/?page=2&type=all",
    "previous": null,
    "results": [
        {
            "type": "RadioStation",
            "value": {
                "slug": "city",
                "name": "Con Vos",
                "webpage": "http://www.radioconvos.com.ar/",
                "audio_url": "https://server1.stweb.tv/rcvos/live/playlist.m3u8",
                "time_zone": "America/Buenos_Aires",
                "logo": "https://imgsvr.radiocut.site/get/crop/center/200/200/radio_logos/77/19/7719d9e2-18d4-4b47-8871-f597b704f75d.jpg",
                "freq": "89.90",
                "listen_mode": "ondemand+live",
                "band": "FM",
                "city": {
                    "code": "BUE",
                    "state": {
                        "code": "AR-C",
                        "country": {
                            "code": "AR",
                            "name": "Argentina"
                        },
      

#### Búsqueda de un término solo en recortes

In [24]:
import requests
import json

url = 'https://radiocut.fm/api/search/?type=cut&search_term=avion'
headers = {
    'Authorization': 'Token ${api_token}'
}
response = requests.get(url)
json_data = response.json()

print(json.dumps(json_data, indent=4, ensure_ascii=False))

{
    "count": 2148,
    "next": "https://radiocut.fm/api/search/?page=2&search_term=avion&type=cut",
    "previous": null,
    "results": [
        {
            "type": "AudioCut",
            "value": {
                "episode": null,
                "description": "Avión estrellado en rosario",
                "title": "Accidente avioneta",
                "radio_obj": {
                    "slug": "rosario2",
                    "name": "Radio 2",
                    "webpage": "http://www.rosario3.com/",
                    "audio_url": "http://radio2.radio.rosario3.com:8000/prueba2",
                    "time_zone": "America/Buenos_Aires",
                    "logo": "https://imgsvr.radiocut.site/get/crop/center/200/200/radio_logos/e3/f6/e3f62861-b5e6-4a21-9b6a-896f92bbb87e.png",
                    "freq": "1230.00",
                    "listen_mode": "ondemand+live",
                    "band": "AM",
                    "city": {
                        "code": "ROS",
       

#### Búsqueda de un término en una radio y creado en los últimos 7 días.

In [26]:
import requests
import json

url = 'https://radiocut.fm/api/search/?type=cut&search_term=avion&radio=am750&created=7'
headers = {
    'Authorization': 'Token ${api_token}'
}
response = requests.get(url)
json_data = response.json()

print(json.dumps(json_data, indent=4, ensure_ascii=False))

{
    "count": 0,
    "next": null,
    "previous": null,
    "results": []
}
